In [203]:
# import external modules
import sys, os
ROOT_PATH = os.path.abspath(".").split("src")[0]
module_path = os.path.abspath(os.path.join(ROOT_PATH+"/src/utils/"))
if module_path not in sys.path:
    sys.path.append(module_path)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate
import warnings
warnings.filterwarnings("ignore")

import functions as f

SEED=100; np.random.seed(100)

In [204]:
# load scaled datasets 
path = ROOT_PATH + "data/dataframes/"
df_complete = pd.read_pickle(path + "df_scaled_complete.pkl")

TRAIN_PERCENTAGE = 0.6         
VALID_PERCENTAGE = 0.2         
TEST_PERCENTAGE = 1.0 - TRAIN_PERCENTAGE - VALID_PERCENTAGE

train_size = int(TRAIN_PERCENTAGE * len(df_complete))
valid_size = int(VALID_PERCENTAGE * len(df_complete))
test_size = len(df_complete) - train_size - valid_size

df_train = df_complete[0:train_size] 
df_valid = df_complete[train_size:train_size+valid_size] 
df_test = df_complete[-test_size:]

In [205]:
# split X and Y. We are using a lookback value for -1 
# Meaning for every input instrument at time T, we are predicting the value of the output instrument at time T+1 minute
def splitXY(dataset,target_columns,lookbackValue=1): 
    assert len(target_columns) > 0, "Please ensure target_columns value are not empty."
    assert lookbackValue > 0, "Please ensure lookbackValue > 0."

    target_columns_lookback = [target+"_lookback" for target in target_columns]
    dataset[target_columns_lookback] = None
    for i in range(len(target_columns_lookback)): 
        dataset[target_columns_lookback[i]][0:-lookbackValue] = dataset[target_columns[i]][lookbackValue:]
    dataset = dataset[0:-lookbackValue].reset_index()
    
    dataset.drop(dataset[target_columns],axis=1,inplace = True)
    renameDict = {} 
    for i in range(len(target_columns)): 
        renameDict[target_columns_lookback[i]] = target_columns[i]
    dataset.rename(columns = renameDict, inplace = True)
    
    x_train = dataset.drop(dataset[target_columns],axis=1)
    x_train.drop(x_train[["timestamp"]],axis=1,inplace=True)

    y_train = dataset[target_columns]
    return x_train,y_train

targetCols = ["VAL_23-FT-92537-01:X.Value","VAL_23-TT-92539:X.Value","VAL_23-PT-92539:X.Value"]
x_train,y_train = splitXY(df_train,targetCols,lookbackValue=1)
x_valid,y_valid = splitXY(df_valid,targetCols,lookbackValue=1)
x_test,y_test = splitXY(df_valid,targetCols,lookbackValue=1)

In [206]:
# load metadata (done by the original author)
path = ROOT_PATH + "data/metadata/"
scaling_stats = pd.read_csv(path+"scaling_stats.csv", index_col=0)
target_tags = y_train.columns
feature_tags = x_train.columns
target_stds = scaling_stats.loc[target_tags,"Std"].values
target_means = scaling_stats.loc[target_tags,"Mean"].values
print(df_train.shape, df_valid.shape, df_test.shape)

(53211, 24) (17737, 24) (17737, 21)


In [207]:
# the tagnames for these instruments are too long. We are shortening them with this function
def shorten_tagname(tag):
    nname = tag.split("VAL_23")[1][1:]
    nname = nname.split(".Value")[0]
    nname = nname.split(":VALUE")[0]
    nname = nname.split(":X")[0]
    
    if "KA_9101" in nname:
        nname = nname.split(":Z.X")[0]
    return nname

short_feature_tags = np.array([shorten_tagname(tag) for tag in feature_tags])
featureTagsShotenDict = {}
for i in range(len(feature_tags)): 
    featureTagsShotenDict[feature_tags[i]] = short_feature_tags[i]

In [208]:
# The feature correlation algorithms 
# Could use boruta 
# Could also use pearson? 
# Kendall’s Rank Correlation
# Pearson’s Correlation Coefficient
# Spearman’s Rank Correlation

In [281]:
# Performing feature importance calculation using linear correlation testing. 
# Reference: https://machinelearningmastery.com/statistical-hypothesis-tests-in-python-cheat-sheet/

from scipy.stats import spearmanr, pearsonr, kendalltau
def calculateCorrelation(corrFunction,xValues,yValues,corrName): 
    xCols = list(xValues.columns)
    yCols = list(yValues.columns)
    corrDataFrame = {}
    for col in yCols: 
        corrDataFrame[col] = None
    
    for ycol in yCols: 
        outputDict = {
        "featureColumn": [],
        "targetColumn":[],
        "score":[],
        "pVal": [],
        "correlationAlgo": []
    }
        yVals = yValues[ycol]
        for xcol in xCols:
            xVals = xValues[xcol]
            stat, p = corrFunction(xVals,yVals)
            outputDict["featureColumn"].append(shorten_tagname(xcol))
            outputDict["targetColumn"].append(ycol)
            outputDict["score"].append(stat)
            outputDict["pVal"].append(p)
            outputDict["correlationAlgo"].append(corrName)
        outputDataFrame = pd.DataFrame(outputDict)
        outputDataFrame["score"] = outputDataFrame["score"].abs()
        outputDataFrame = outputDataFrame.loc[abs(outputDataFrame.pVal) < 0.05]
        outputDataFrame.sort_values("score", axis=0, ascending=False,inplace=True)
        corrDataFrame[ycol] = outputDataFrame

    return corrDataFrame

spearmanCorr = calculateCorrelation(spearmanr,x_train,y_train,"spearmanr")
pearsonCorr = calculateCorrelation(pearsonr,x_train,y_train,"pearson")
kendallCorr = calculateCorrelation(kendalltau,x_train,y_train,"kendall")

In [282]:
ycols = list(y_train.columns)
spearman_FT_92537 = spearmanCorr[ycols[0]][:10]
pearson_FT_92537  = pearsonCorr[ycols[0]][:10]
kendall_FT_92537  = kendallCorr[ycols[0]][:10]

spearman_TT_92539 = spearmanCorr[ycols[1]][:10]
pearson_TT_92539  = pearsonCorr[ycols[1]][:10]
kendall_TT_92539  = kendallCorr[ycols[1]][:10]

spearman_PT_92539 = spearmanCorr[ycols[2]][:10]
pearson_PT_92539  = pearsonCorr[ycols[2]][:10]
kendall_PT_92539  = kendallCorr[ycols[2]][:10]


In [283]:
spearman_FT_92537_names = list(spearmanCorr[ycols[0]][:10]["featureColumn"])
pearson_FT_92537_names  = list(pearsonCorr[ycols[0]][:10]["featureColumn"])
kendall_FT_92537_names  = list(kendallCorr[ycols[0]][:10]["featureColumn"])

spearman_TT_92539_names = list(spearmanCorr[ycols[1]][:10]["featureColumn"])
pearson_TT_92539_names  = list(pearsonCorr[ycols[1]][:10]["featureColumn"])
kendall_TT_92539_names  = list(kendallCorr[ycols[1]][:10]["featureColumn"])

spearman_PT_92539_names = list(spearmanCorr[ycols[2]][:10]["featureColumn"])
pearson_PT_92539_names  = list(pearsonCorr[ycols[2]][:10]["featureColumn"])
kendall_PT_92539_names  = list(kendallCorr[ycols[2]][:10]["featureColumn"])

In [284]:
# Checking whcih features are most important for FT output
commonTags_FT_92537 = []
for tag in spearman_FT_92537_names: 
    if tag in pearson_FT_92537_names and tag in kendall_FT_92537_names:
        commonTags_FT_92537.append(tag)
commonTags_FT_92537

['PDT-92534',
 'KA_9101_M01_62C',
 'KA-9101_ASP',
 'FT-92521',
 'LIC-92521:Z.Y',
 'PDT-92522',
 'TIC-92504:Z.X',
 'TT-92512',
 'PT-92512',
 'ZT_92543:Z.X']

In [285]:
# Checking whcih features are most important for TT output
commonTags_TT_92539 = []
for tag in spearman_TT_92539_names: 
    if tag in pearson_TT_92539_names and tag in kendall_TT_92539_names:
        commonTags_TT_92539.append(tag)
commonTags_TT_92539

['TT_92532:Z.X',
 'ZT_92543:Z.X',
 'FT-92512',
 'PDT-92502',
 'LIC-92521:Z.Y',
 'TT-92512',
 'FT-92521',
 'PT-92512',
 'KA-9101_ESP',
 'VA-9110-M01-EL']

In [286]:
# Checking whcih features are most important for PT output 
commonTags_PT_92539 = []
for tag in spearman_PT_92539_names: 
    if tag in pearson_PT_92539_names and tag in kendall_PT_92539_names:
        commonTags_PT_92539.append(tag)
commonTags_PT_92539

['PT-92523',
 'KA-9101_ESP',
 'PT-92512',
 'KA_9101_M01_62C',
 'KA-9101_ASP',
 'TT_92532:Z.X',
 'FT-92512',
 'TIC-92504:Z.X',
 'FT-92521',
 'LIC-92521:Z.X']

In [298]:
spearmanFT = spearmanCorr[ycols[0]][["featureColumn","score"]] 
spearmanTT = spearmanCorr[ycols[1]][["featureColumn","score"]]
spearmanPT = spearmanCorr[ycols[2]][["featureColumn","score"]]

merged3_spearman = spearmanFT.merge(spearmanTT, on='featureColumn', how='left')
merged3_spearman = merged3_spearman.merge(spearmanPT, on='featureColumn', how='left')
merged3_spearman.columns = ["featureColumn","FT","TT","PT"]
merged3_spearman.dropna(axis = 0,inplace = True)
merged3_spearman["averageSpearman"] = (merged3_spearman["FT"]+merged3_spearman["TT"]+merged3_spearman["PT"]) / 3 
merged3_spearman.sort_values("averageSpearman",ascending=False,inplace = True)
merged3_spearman = merged3_spearman[:10]
merged3_spearman

,featureColumn,FT,TT,PT,averageSpearman
8,PT-92512,0.205671,0.234454,0.610229,0.350118
17,TT_92532:Z.X,0.029532,0.819830,0.197172,0.348845
9,ZT_92543:Z.X,0.167392,0.728701,0.078159,0.324750
16,KA-9101_ESP,0.032373,0.202044,0.669116,0.301178
1,KA_9101_M01_62C,0.380740,0.052007,0.464555,0.299101
13,PT-92523,0.076321,0.091579,0.689879,0.285926
15,FT-92512,0.036031,0.634652,0.127643,0.266109
3,FT-92521,0.319052,0.285168,0.094367,0.232862
2,KA-9101_ASP,0.335724,0.019193,0.323428,0.226115
14,PDT-92502,0.066853,0.537335,0.059896,0.221362


In [300]:
pearsonFT = pearsonCorr[ycols[0]][["featureColumn","score"]] 
pearsonTT = pearsonCorr[ycols[1]][["featureColumn","score"]]
pearsonPT = pearsonCorr[ycols[2]][["featureColumn","score"]]

merged3_pearson = pearsonFT.merge(pearsonTT, on='featureColumn', how='left')
merged3_pearson = merged3_pearson.merge(pearsonPT, on='featureColumn', how='left')
merged3_pearson.columns = ["featureColumn","FT","TT","PT"]
merged3_pearson.dropna(axis = 0,inplace = True)
merged3_pearson["averagePearson"] = (merged3_pearson["FT"]+merged3_pearson["TT"]+merged3_pearson["PT"]) / 3 
merged3_pearson.sort_values("averagePearson",ascending=False,inplace = True)
merged3_pearson = merged3_pearson[:10]

,featureColumn,FT,TT,PT,averagePearson
9,PT-92512,0.204568,0.238850,0.621116,0.354845
8,ZT_92543:Z.X,0.218124,0.775352,0.067461,0.353646
13,FT-92512,0.109280,0.704706,0.115285,0.309757
14,PT-92523,0.053352,0.096899,0.688745,0.279665
16,KA-9101_ESP,0.032836,0.188588,0.577158,0.266194
15,PDT-92502,0.035344,0.596387,0.049632,0.227121
6,FT-92521,0.255139,0.297949,0.104770,0.219286
7,TT-92512,0.249594,0.390860,0.016871,0.219108
2,KA-9101_ASP,0.302618,0.049847,0.297398,0.216621
0,PDT-92534,0.452858,0.081294,0.046003,0.193385


In [ ]:
kendallFT = kendallCorr[ycols[0]][["featureColumn","score"]] 
kendallTT = kendallCorr[ycols[1]][["featureColumn","score"]]
kendallPT = kendallCorr[ycols[2]][["featureColumn","score"]]

merged3_kendall = kendallFT.merge(kendallTT, on='featureColumn', how='left')
merged3_kendall = merged3_kendall.merge(kendallPT, on='featureColumn', how='left')
merged3_kendall.columns = ["featureColumn","FT","TT","PT"]
merged3_kendall.dropna(axis = 0,inplace = True)
merged3_kendall["averageKendall"] = (merged3_kendall["FT"]+merged3_kendall["TT"]+merged3_kendall["PT"]) / 3 
merged3_kendall.sort_values("averageKendall",ascending=False,inplace = True)
merged3_kendall = merged3_kendall[:10]

In [308]:
merged3_spearman_names = list(merged3_spearman.featureColumn)
merged3_pearson_names = list(merged3_pearson.featureColumn)
merged3_kendall_names = list(merged3_kendall.featureColumn)
intersectAverage = []
for tag in merged3_spearman_names: 
    if tag in merged3_pearson_names and tag in merged3_kendall_names: 
        intersectAverage.append(tag)

intersectFinal = []
for tag in intersectAverage:
    if tag in commonTags_FT_92537 and tag in commonTags_TT_92539 and tag in commonTags_PT_92539:
        intersectFinal.append(tag)
intersectAverage

['PT-92512',
 'ZT_92543:Z.X',
 'KA-9101_ESP',
 'PT-92523',
 'FT-92512',
 'FT-92521',
 'KA-9101_ASP',
 'PDT-92502']